## Normalization

### 2NF - Second Normal Form
- It should be in the `1nf`
- Check the different anomalies: `Deletion`, `Insertion`, `Update`
    - `Deletion Anomaly`:
        > Deletion of one thing, leads to the unintentional loss of data.
    - `Insertion Anomaly`:
        > Data can't be inserted because of lack of necessary information.
    - `Update Anomaly`:
        > A fail in the update will cause inconsistent data in the database.

> It says, each non-key attribute must depend on the entire primary key.

- From the table
    - If you know the `reader_username`, then you can determine the `title`, `first name`, `last name`. In fact, all information about the reader.
    - If you know the `book_isbn`, then you can determine all information about that book

When we separate into many tables, we have
- `book`: with primary key `book_isbn`
- `reader`: with primary key `reader_username`
- `myread`: with surrogate primary key `id`.

### 3NF - Third Normal Form
> It says a non-key attribute shouldn't depend on another non-key attribute, but on the entire primary key and nothing but the entire primary key.

This normal form tries to avoid `transitive dependency`. When a non-key attribute depends on another non-key attribute.

- For example, you have a table `employee_status
    - `emp_id PRIMARY KEY`
    - `skill_level`
    - `seniority`
```
emp_id   skill_level   seniority
1           10           senior
2           10           senior
3           5            mid-level
```

**employ_status**

|emp_id|skill_level|
|------|-----------|
|1     |  10       |
|2     |  10       |
|3     |  5        |


**skill_seniority**

|skill_level|seniority|
|-----------|---------|
|1,2,3      | Beginner |
|4,5,6      | Mid-level|
|7,8,9,10   | Senior   |

- In our `myread` table, we have
    - `read_status` That will be either `pending`, `reading`, `done`.
    - `percentage_read`: This describe the percentage read. 0%, 10%, 100%

**read_status_percentage**

|percentage_range|read_status|
|----------------|-----------|
|[0,0]           | pending   |
|[1,99]          | reading   |
|[100,100]       | done      |

### Ideas for future improvement -> Exercise
- let the system auto-calculate this percentage based on the book's page count and the reader read page count.

## Foreign Key
- We need a column from the book to represent books in the `myread`
- We need a column from the reader to represent readers in the `myread`.

**NB**: The best candidate for foreign is the primary key. It could be a column with the UNIQUE constraint.

- `book` -> `book_isbn`
- `reader` -> `reader_username`

## Define data types and business rules (constraints)

**book**

- `isbn`: CHAR(13) PRIMARY KEY CHECK(LENGTH(isbn) = 13 AND isbn::BIGINT = isbn::BIGINT)
- `title`: VARCHAR(50) NOT NULL
- `edition`: INT
- `description`: TEXT
- `page_count`: INT NOT NULL
- `category`: ENUM('programming', 'art', 'politics', 'others')
- `published_date`: DATE NOT NULL
- `publisher`: VARCHAR(50) NOT NULL
- `authors`: VARCHAR(50) ARRAY
- `lang`: VARCHAR(10) NOT NULL
- `format`: VARCHAR(10) CHECK(format IN ('ebook', 'hardcover'))
- `read_estimated_time_in_minutes`: INT GENERATED ALWAYS AS ((page_count * 120)/60) VIRTUAL | STORED

**reader**

- `username`: VARCHAR(50) PRIMARY KEY
- `title`: ENUM(Mrs, Mr, Dr, Ms, Miss)
- `first_name`: VARCHAR(100) NOT NULL
- `last_name`: VARCHAR(100)

**status_percent**

- `read_status`: VARCHAR(10) PRIMARY KEY
- `percentage_read`: INT4RANGE NOT NULL


**myread**
- `id` SERIAL PRIMARY KEY
- `book_isbn`: CHAR(13) FOREIGN KEY REFERENCES book(isbn)
- `reader_username`: VARCHAR(50) FOREIGN KEY REFERENCES reader(username)
- `start_read_date`: DATE 
- `end_read_date`: DATE
- `percentage_read`: INT

### Exercise - Constraints for the myread table
- The `end_read_date` should be ahead of `start_read_date` in time
- The `percentage_read` should be between `0` and `100` inclusively
- `percentage_read` shouldn't be `0` if `start_read_date` is set
- `percentage_read` should be `0` if `start_read_date` is not set
- `percentage_read` should be `100` when `end_read_date` is set.
- `percentage_read` shouldn't be `100` if `end_read_date` is not set.



## Query multiple tables
### Exercise
1. Select the `username`, `first_name` and `last_name` of all readers with 100% `percentage_read`

```sql
SELECT DISTINCT(username), first_name, last_name 
FROM reader, my_read
WHERE percentage_read = 100;
```

2. Select the `username`, `names`(where names is the concatenation of `first_name` and `last_name` ), `published_date` of readers reading books published in 2020

```python
first_name = 'kevin'
last_name = 'eyong
names = first_name + ' ' + last_name
```
```sql
-- reader
-- book
-- my_read
-- concatenate with ||

SELECT DISTINCT(username), first_name || ' ' || last_name AS names, published_date
FROM reader, book, my_read
WHERE 
    -- all books were checking are read
    book.isbn = my_read.book_isbn
    -- the book was published in 2020
    AND book.published_date = 2021;
```

## JOINS
- The process known as `table join` allows us to link rows in one table to rows in another tables.
- To connect tables in a query, we can either do it with `JOIN...ON` or `JOIN...USING` statements.

- Common Syntax

```sql
SELECT <column-list-from-table-a-and-b>
FROM table_a JOIN table_b
ON table_a.pkey_column = table_b.foreign_key_colum
[WHERE condition];
```

### Exercise - Using JOIN...ON
1. Select the `username`, `first_name` and `last_name` of all readers with 100% `percentage_read`

```sql
SELECT DISTINCT(username), first_name, last_name 
FROM reader, my_read
WHERE 
    username = reader_username AND
    percentage_read = 100;

-- Using JOIN...ON
SELECT DISTINCT(username), first_name, last_name
FROM reader INNER JOIN my_read
ON username = reader_username
WHERE percentage_read = 100;
```

### Some Convention

```sql
/*
No particular order, but by convention, we use the following

FROM <PK-table> JOIN <FK-table>
ON <PK-table>.<pk> = <FK-table>.<fk>
*/
```

### USING
- It is common for the PK and FK to have the same name
- It is common to set the basis of the join on the `quality` comparison operator. `=`

```sql

ALTER TABLE my_read RENAME COLUMN reader_username TO username;
ALTER TABLE my_read RENAME COLUMN book_isbn TO isbn;

SELECT DISTINCT(username), first_name, last_name
FROM reader JOIN my_read
USING(username) -- ON reader.username = my_read.username
WHERE percentage_read = 100;
```

## JOIN TYPES
We have four different types of JOINS
- INNER 
- OUTER
- CROSS
- SELF

### INNER JOIN or JOIN
> Returns rows from both tables where matching values are found in the joined columns of both tables.

**When to use**
- When you want to return only records having pair on both side

## OUTER JOINS
## LEFT JOIN or LEFT OUTER JOIN
> Return rows from the left table, plus rows that match values in the joined column from the right table. If the left table doesn't have any match in the right table, the results shows no values from the right table by displaying blank rows or whatever NULL was set to. `\pset NULL ''`

- Left table is prioritized
- Right table is less-prioritized

**Example**: Select isbn and username from the book and my_read respectively

```sql
SELECT DISTINCT(book.isbn), username
FROM book LEFT JOIN my_read
USING(isbn);
```

## RIGHT JOIN or RIGHT OUTER JOIN
Return rows from the RIGHT table, plus rows that match values in the joined column from the LEFT table. If the RIGHT table doesn't have any match in the LEFT table, the results shows no values from the LEFT table by displaying blank rows or whatever NULL was set to. `\pset NULL ''`

- RIGHT table is prioritized
- LEFT table is less-prioritized

```sql
SELECT DISTINCT(book.isbn), username
FROM book RIGHT JOIN my_read
USING(isbn);
```

**When to use**
- When you want to find missing rows in one of the tables you are joining.
- When we want to find rows that exist in one or few tables among all the tables we are joining.

```sql
SELECT DISTINCT(book.isbn), username
FROM book LEFT JOIN my_read
USING(isbn)
WHERE username IS NULL;
```

## FULL OUTER JOIN or FULL JOIN

```sql
SELECT DISTINCT(book.isbn), username
FROM book FULL JOIN my_read
USING(isbn);

-- CREATE TABLE reader_identification 

CREATE TABLE reader_identification(
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE REFERENCES reader(username),
    identification_type VARCHAR(30) DEFAULT 'email',

    CHECK(identification_type IN ('email', 'phone'))
);

INSERT INTO reader_identification (username, identification_type)
    VALUES('agerrens6', 'email'),
        ('tdefraine7', 'email'),
        ('estoeckle8', 'phone'),
        ('pcarvil9', 'phone');


SELECT DISTINCT(my_read.username) reader_username, reader_identification.username identified_username
FROM my_read FULL JOIN reader_identification
USING(username);
```

## CROSS JOIN

> Returns every possible combination of rows from both tables.

- Cartesian product.
    - table a -> `n` rows
    - table b -> `m` rows
    - = `n * m`

**When to use**
- We want to send specific products to our warehouses in all location.

#### Exercise - Create tables `shirt_model` and `shirt_color` and try the `CROSS JOIN`
- Shirts
    - table a: contains different models of shirts
        - model a
        - model b
    - table b: contains different colors
        - red
        - yellow
        - white
    - Output
        - model a, red
        - model a, yellow
        - model a, white
        - model b, red
        - model b, yellow
        - model b, white.

**When not to use**
- Since it returns the Cartesian product, it is not advisable to use it in large tables due to the size of its output.


## SELF JOINS

```sql
table_a JOIN table_a
```

### Church Hierarchy table
- God, Pope, Cardinal, Bishop, Priest

```
supervisor -> worker
God        -> Pope
Pope       -> Cardinal
Cardinal   -> Bishop
Bishop     -> Priest
```

```sql
CREATE TABLE church_hierarchy(
    id SERIAL PRIMARY KEY,   -- 2
    position_id SERIAL,      -- 2
    supervisor_id INT,       -- 1
    position_des VARCHAR(100) -- Pope
);

INSERT INTO church_hierarchy(position_des, supervisor_id)
    VALUES ('God', null),
            ('Pope', 1),
            ('Cardinal', 2),
            ('Bishop', 3),
            ('Priest', 4);

SELECT w.position_des || ' -> ' || s.position_des AS "Supervisor -> Worker"
FROM church_hierarchy w
JOIN church_hierarchy s
ON  s.supervisor_id = w.position_id;
```

## Joining more than two tables

```sql
SELECT <column-list>
FROM <pk-table>
    JOIN <fk-pk-table> ON ...
        JOIN <fk-table> ON ...
```
**Exercise**: Select username, first name, read status of all readers with read status 'pending'

```sql
-- reader, my_read, status_percent

SELECT DISTINCT(reader.username), first_name, read_status
FROM reader
    JOIN my_read USING(username)
        JOIN status_percent ON my_read.percentage_read <@ status_percent.percentage_read
WHERE read_status = 'pending';
```
